In [1]:
from block_sync_class import *
from bokeh.plotting import figure, show
import pickle
import time

In [2]:

block_0 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='0',
                    path_to_animal_folder=r'D:\experiments\\')
block_1 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='1',
                    path_to_animal_folder=r'D:\experiments\\')
block_2 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='2',
                    path_to_animal_folder=r'D:\experiments\\')
block_3 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='3',
                    path_to_animal_folder=r'D:\experiments\\')
block_4 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='4',
                    path_to_animal_folder=r'D:\experiments\\')
block_5 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='5',
                    path_to_animal_folder=r'D:\experiments\\')
block_6 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='6',
                    path_to_animal_folder=r'D:\experiments\\')
block_7 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='7',
                    path_to_animal_folder=r'D:\experiments\\')

blocklist = [
    block_0,
    block_1,
    block_2,
    block_3,
    block_4,
    block_5,
    block_6,
    block_7
]

instantiated block number 0 at Path: D:\experiments\Animal_4\18_07_2021\block_0
instantiated block number 1 at Path: D:\experiments\Animal_4\18_07_2021\block_1
instantiated block number 2 at Path: D:\experiments\Animal_4\18_07_2021\block_2
instantiated block number 3 at Path: D:\experiments\Animal_4\18_07_2021\block_3
instantiated block number 4 at Path: D:\experiments\Animal_4\18_07_2021\block_4
instantiated block number 5 at Path: D:\experiments\Animal_4\18_07_2021\block_5
instantiated block number 6 at Path: D:\experiments\Animal_4\18_07_2021\block_6
instantiated block number 7 at Path: D:\experiments\Animal_4\18_07_2021\block_7


In [ ]:
for block in blocklist:
    block.handle_arena_files()
    block.handle_eye_videos()

In [ ]:
    block.synchronize_arena_timestamps()

    block.create_arena_brightness_df(250, export=True)

In [ ]:
for block in blocklist:


In [ ]:
for block in blocklist:
    block.synchronize_arena_timestamps()

In [ ]:
for block in blocklist:
    block.import_arena_brightness_file()

In [ ]:
for block in blocklist:
    start = time.time()
    block.create_arena_brightness_df(threshold_value=255, export=True)
    end = time.time()
    print(f'this one took {end - start} seconds')

In [ ]:
for block in blocklist:
    #block.validate_arena_synchronization()
    block.import_open_ephys_events()
    block.synchronize_block()
    block.update_arena_bdf()
    block.create_eye_brightness_df(threshold_value=30)
    block.synchronize_by_led_blink()
    
    file_to_save_to = open(block.block_path / 'BlockSync.pickle','wb')
    pickle.dump(block, file_to_save_to)
    file_to_save_to.close()
    print(f'block number {block.block_num} saved')

In [ ]:
block_0 = BlockSync(animal_num='4',
                    experiment_date='18_07_2021',
                    block_num='2',
                    path_to_animal_folder=r'D:\experiments\\')

In [ ]:
block_0.handle_arena_files()

In [ ]:
block_0.handle_eye_videos()

In [ ]:
block_0.synchronize_arena_timestamps()


In [ ]:
block_0.create_arena_brightness_df(250, export=True)

In [ ]:
block_0.import_arena_brightness_file()

In [ ]:
block_0.arena_brightness_df

In [ ]:
block_7.validate_arena_synchronization()


In [ ]:
block_7.import_open_ephys_events()

In [ ]:
block_7.synchronize_block()

In [ ]:
block_7.update_arena_bdf()

In [ ]:
block_7.create_eye_brightness_df(30)

In [ ]:
block_7.synchronize_by_led_blink()

In [ ]:
block_4.manual_sync(30)

In [ ]:
block_0.block_num

In [ ]:
file_to_save_to = open(block_7.block_path / 'BlockSync.pickle','wb')
pickle.dump(block_7,file_to_save_to)
file_to_save_to.close()

In [ ]:
block_7.create_synced_videos_validated()
block_7.export_everything()

In [ ]:
##### PHASE 2 #####

In [7]:
def eye_tracking_analysis_v2(dlc_video_analysis_csv,uncertainty_thr):
    """
    :param dlc_video_analysis_csv: the csv output of a dlc analysis of one video, already read by pandas with header=1
    :param uncertainty_thr: The confidence P value to use as a threshold for datapoint validity in the analysis
    :returns ellipse_df: a DataFrame of ellipses parameters (center, width, height, phi, size) for each video frame

    """
# import the dataframe and convert it to floats
    data = dlc_video_analysis_csv
    data = data.iloc[1:].apply(pd.to_numeric)
# sort the pupil elements to x and y, with p as probability
    pupil_elements = np.array([x for x in data.columns if 'Pupil' in x])
    pupil_xs = data[pupil_elements[np.arange(0,len(pupil_elements),3)]]
    pupil_ys = data[pupil_elements[np.arange(1,len(pupil_elements),3)]]
    pupil_ps = data[pupil_elements[np.arange(2,len(pupil_elements),3)]]
    #rename dataframes for masking with p values of bad points:
    pupil_ps = pupil_ps.rename(columns=dict(zip(pupil_ps.columns,pupil_xs.columns)))
    pupil_ys = pupil_ys.rename(columns=dict(zip(pupil_ys.columns,pupil_xs.columns)))
    good_points = pupil_ps > uncertainty_thr
    pupil_xs = pupil_xs[good_points]
    pupil_ys = pupil_ys[good_points]
    #Do the same for the edges
    #edge_elements = [x for x in data.columns if 'edge' in x]
    #edge_xs = data[edge_elements[np.arange(0,len(edge_elements),3)]]
    #edge_ys = data[edge_elements[np.arange(1,len(edge_elements),3)]]
    #edge_ps = data[edge_elements[np.arange(2,len(edge_elements),3)]]
    #edge_ps = edge_ps.rename(columns=dict(zip(edge_ps.columns,edge_xs.columns)))
    #edge_ys = edge_ys.rename(columns=dict(zip(edge_ys.columns,edge_xs.columns)))
    #e = edge_ps < uncertainty_thr


    #work row by row to figure out the ellipses
    ellipses = []
    caudal_edge_ls= []
    rostral_edge_ls = []
    for row in tqdm(range(1, len(data)-1)):
        #first, take all of the values, and concatenate them into an X array
        x_values = pupil_xs.loc[row].values
        y_values = pupil_ys.loc[row].values
        X = np.c_[x_values,y_values]
        #now, remove nan values, and check if there are enough points to make the ellipse
        X = X[~ np.isnan(X).any(axis=1)]
        #if there are enough rows for a fit, make an ellipse
        if X.shape[0] > 5:
            el = LsqEllipse().fit(X)
            center, width, height, phi = el.as_parameters()
            center_x = center[0]
            center_y = center[1]
            ellipses.append([center_x,center_y,width,height,phi])
        else:
            ellipses.append([np.nan, np.nan, np.nan, np.nan, np.nan])
        # caudal_edge = [
        #     float(data['Caudal_edge'][row]),
        #     float(data['Caudal_edge.1'][row])
        # ]
        # rostral_edge = [
        #     float(data['Rostral_edge'][row]),
        #     float(data['Rostral_edge.1'][row])
        # ]
        # caudal_edge_ls.append(caudal_edge)
        # rostral_edge_ls.append(rostral_edge)
        #if row % 50 == 0:
        #   print(f'just finished with {row} out of {len(data)-1}', end='\r',flush=True)
    ellipse_df = pd.DataFrame(columns = ['center_x','center_y', 'width', 'height', 'phi'], data = ellipses)
    a = np.array(ellipse_df['height'][:])
    b = np.array(ellipse_df['width'][:])
    ellipse_size_per_frame = a*b*math.pi
    ellipse_df['ellipse_size'] = ellipse_size_per_frame
    #ellipse_df['rostral_edge'] = rostral_edge_ls
    #ellipse_df['caudal_edge'] = caudal_edge_ls
    print('\n Done')
    return ellipse_df

In [8]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
from ellipse import LsqEllipse
import math
from bokeh.plotting import figure, show

In [ ]:
for block_num in ['0','1','2','3','5','6','7']:
    animal_number = '4'
    experiment_date = '18_07_2021'
    block = block_num
    path = rf'D:\experiments\Animal_{animal_number}\{experiment_date}\block_{block}'
    print(path)
    le_csv = pd.read_csv(glob.glob(path + r'\eye_videos\LE\**\*DLC*.csv')[0],header=1)
    re_csv = pd.read_csv(glob.glob(path + r'\eye_videos\RE\**\*DLC*.csv')[0],header=1)

    le_ellipses = eye_tracking_analysis_v2(le_csv, 0.999)
    print('\n')
    re_ellipses = eye_tracking_analysis_v2(re_csv, 0.999)
    video_sync_df = pd.read_csv(path + r'\analysis\synced_videos_validated.csv')
    le_video_sync_df = video_sync_df.drop(labels=['Arena_TTL','R_eye_TTL'],axis=1)
    for column in list(le_ellipses.columns):
        le_video_sync_df.insert(loc=len(le_video_sync_df.columns), column=column, value=None)
    re_video_sync_df = video_sync_df.drop(labels=['Arena_TTL','L_eye_TTL'],axis=1)
    for column in list(re_ellipses.columns):
        re_video_sync_df.insert(loc=len(re_video_sync_df.columns), column=column, value=None)
    print('populating le_video_sync_df')
    for row in tqdm(le_video_sync_df.index):
        frame = le_video_sync_df.L_eye_TTL[row]
        le_video_sync_df.loc[row, 'center_x'] = le_ellipses.iloc[frame]['center_x']
        le_video_sync_df.loc[row, 'center_y'] = le_ellipses.iloc[frame]['center_y']
        le_video_sync_df.loc[row, 'width']= le_ellipses.width[frame]
        le_video_sync_df.loc[row, 'height'] = le_ellipses.height[frame]
        le_video_sync_df.loc[row, 'phi'] = le_ellipses.phi[frame]
        le_video_sync_df.loc[row, 'ellipse_size'] = le_ellipses.ellipse_size[frame]
        #le_video_sync_df.at[row, 'rostral_edge'] = le_ellipses.rostral_edge[frame]
        #le_video_sync_df.at[row, 'caudal_edge'] = le_ellipses.caudal_edge[frame]
    print('populating re_video_sync_df')
    for row in tqdm(re_video_sync_df.index):
        frame = re_video_sync_df.R_eye_TTL[row]
        re_video_sync_df.loc[row, 'center_x'] = re_ellipses.iloc[frame]['center_x']
        re_video_sync_df.loc[row, 'center_y'] = re_ellipses.iloc[frame]['center_y']
        re_video_sync_df.loc[row, 'width']= re_ellipses.width[frame]
        re_video_sync_df.loc[row, 'height'] = re_ellipses.height[frame]
        re_video_sync_df.loc[row, 'phi'] = re_ellipses.phi[frame]
        re_video_sync_df.loc[row, 'ellipse_size'] = re_ellipses.ellipse_size[frame]
        #re_video_sync_df.at[row, 'rostral_edge'] = re_ellipses.rostral_edge[frame]
        #re_video_sync_df.at[row, 'caudal_edge'] = re_ellipses.caudal_edge[frame]

    first_frame_time = le_video_sync_df.Time.values[0]
    le_video_sync_df

    le_ellipses_z = (le_video_sync_df.ellipse_size - le_video_sync_df.ellipse_size.mean()) / le_video_sync_df.ellipse_size.std()
    re_ellipses_z = (re_video_sync_df.ellipse_size - re_video_sync_df.ellipse_size.mean()) / re_video_sync_df.ellipse_size.std()
    le_x_zscores = (le_video_sync_df.center_x - np.mean(le_video_sync_df.center_x)) / le_video_sync_df.center_x.std()
    le_y_zscores = (le_video_sync_df.center_y - np.mean(le_video_sync_df.center_y)) / le_video_sync_df.center_y.std()
    re_x_zscores = (re_video_sync_df.center_x - np.mean(re_video_sync_df.center_x)) / re_video_sync_df.center_x.std()
    re_y_zscores = (re_video_sync_df.center_y - np.mean(re_video_sync_df.center_y)) / re_video_sync_df.center_y.std()
    x_axis = le_video_sync_df.Time[range(len(le_ellipses_z))] - first_frame_time

    bokeh_fig = figure(title='Pupil combined metrics : '+str(path),
                       x_axis_label='Record Time',
                       y_axis_label=' Z scores',
                       plot_width=1500,
                       plot_height=700
                       )

    bokeh_fig.line(x_axis, le_ellipses_z + 7, legend_label='Left Eye diameter', line_width=1.5, line_color='blue')
    bokeh_fig.line(x_axis, le_x_zscores + 14, legend_label='Left Eye x position', line_width=1, line_color='cyan')
    bokeh_fig.line(x_axis, le_y_zscores, legend_label='Left Eye y position', line_width=1, line_color='green')
    bokeh_fig.line(x_axis, re_ellipses_z + 7, legend_label='Right Eye diameter', line_width=1.5, line_color='red')
    bokeh_fig.line(x_axis, re_x_zscores + 14, legend_label='Right Eye x position', line_width=1, line_color='orange')
    bokeh_fig.line(x_axis, re_y_zscores, legend_label='Right Eye y position', line_width=1, line_color='pink')
    #bokeh_fig.line(x_axis, np.gradient(re_ellipses_z), legend_label='RE Gradient', line_width=1, line_color='green')
    show(bokeh_fig)



In [3]:
for ind, block in enumerate(blocklist):
    if block.block_num == '4':
        print(block.block_num)
        print(ind)
    else:
        block.import_everything()

4
4


In [4]:
blocklist.pop(4)

BlockSync object with block_num 4

In [5]:
for block in blocklist:
    block.read_dlc_data()

100%|██████████| 13741/13741 [01:06<00:00, 206.89it/s]



 Done

 Done
populating le_video_sync_df
populating re_video_sync_df

 Done

 Done
populating le_video_sync_df
populating re_video_sync_df

 Done

 Done
populating le_video_sync_df
populating re_video_sync_df

 Done

 Done
populating le_video_sync_df
populating re_video_sync_df

 Done

 Done
populating le_video_sync_df
populating re_video_sync_df

 Done

 Done
populating le_video_sync_df
populating re_video_sync_df

 Done

 Done
populating le_video_sync_df
populating re_video_sync_df


In [9]:
generator = block_0.block_path.glob(r'eye_videos\LE\**\*DLC*.csv')

In [10]:
re_ellipses_0 = block_0.re_ellipses
re_ellipses_1 = block_1.re_ellipses
df = pd.concat([re_ellipses_0, re_ellipses_1],keys=['block_0','block_1'])
df

center_x    center_y      width     height       phi  \
block_0 0      260.179857  185.501207  43.166207  39.919914  0.053955   
        1      260.293307  185.506411  43.082061  39.815916  0.052303   
        2      260.274120  185.511999  43.046617  39.758611  0.052419   
        3      260.138567  185.498857  43.061174  39.785922  0.067378   
        4      260.214086  185.561911  43.056265  39.867469  0.047372   
...                   ...         ...        ...        ...       ...   
block_1 14441  233.271468  198.214537  45.334997  41.042013 -0.470627   
        14442  233.431119  198.126628  45.379001  41.025202 -0.492387   
        14443  233.468391  198.109616  45.336925  41.005348 -0.456247   
        14444  233.344425  198.008292  45.301183  41.103743 -0.466260   
        14445  233.580870  197.999173  45.289935  41.055390 -0.459757   

               ellipse_size  
block_0 0       5413.564929  
        1       5388.936285  
        2       5376.753173  
        3       5382.266070  
        4       5392.683006  
...                     ...  
block_1 14441   5845.371452  
        14442   5848.648563  
        14443   5840.397861  
        14444   5849.796848  
        14445   5841.464558  

[29293 rows x 6 columns]

In [13]:
re_dfs_to_concat = []
le_dfs_to_concat =[]
keys = []
breaks = [0]
for block in blocklist:
    keys.append(f'block_{block.block_num}')
    re_dfs_to_concat.append(block.re_video_sync_df)
    le_dfs_to_concat.append(block.le_video_sync_df)
    breaks.append(len(block.re_video_sync_df)+breaks[-1])

In [14]:
re_mega_df = pd.concat(re_dfs_to_concat, keys=keys)
le_mega_df = pd.concat(le_dfs_to_concat, keys=keys)

In [15]:
breaks

[0, 13741, 27482, 41225, 50046, 61389, 73930, 87671]

In [66]:
#Extract the data columns and calculate Z-score across all blocks
le_ellipses_z = (le_mega_df.ellipse_size - le_mega_df.ellipse_size.mean()) / le_mega_df.ellipse_size.std()
re_ellipses_z = (re_mega_df.ellipse_size - re_mega_df.ellipse_size.mean()) / re_mega_df.ellipse_size.std()
le_x_zscores = (le_mega_df.center_x - np.mean(le_mega_df.center_x)) / le_mega_df.center_x.std()
le_y_zscores = (le_mega_df.center_y - np.mean(le_mega_df.center_y)) / le_mega_df.center_y.std()
re_x_zscores = (re_mega_df.center_x - np.mean(re_mega_df.center_x)) / re_mega_df.center_x.std()
re_y_zscores = (re_mega_df.center_y - np.mean(re_mega_df.center_y)) / re_mega_df.center_y.std()
x_axis = range(len(le_ellipses_z))

In [67]:
# plot everything
bokeh_fig = figure(title='Pupil combined metrics',
                   x_axis_label='Linear Frames',
                   y_axis_label=' Z scores',
                   plot_width=1500,
                   plot_height=700
                   )

#bokeh_fig.line(x_axis, le_ellipses_z + 7, legend_label='Left Eye diameter', line_width=1.5, line_color='blue')
#bokeh_fig.line(x_axis, le_x_zscores + 14, legend_label='Left Eye x position', line_width=1, line_color='cyan')
#bokeh_fig.line(x_axis, le_y_zscores, legend_label='Left Eye y position', line_width=1, line_color='green')
bokeh_fig.line(x_axis, re_ellipses_z + 7, legend_label='Right Eye diameter', line_width=1.5, line_color='red')
bokeh_fig.line(x_axis, re_x_zscores + 14, legend_label='Right Eye x position', line_width=1, line_color='orange')
bokeh_fig.line(x_axis, re_y_zscores, legend_label='Right Eye y position', line_width=1, line_color='pink')
bokeh_fig.cross(breaks, [-5,-5,-5,-5,-5,-5,-5,-5], size=20, color="navy", alpha=0.5)
#bokeh_fig.line(x_axis, np.gradient(re_ellipses_z), legend_label='RE Gradient', line_width=1, line_color='green')
show(bokeh_fig)

In [ ]:
# parse it

# cross-correlate all to all
# dendrogram